In [5]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.arima_process import ArmaProcess
from matplotlib import pyplot as plt
from scipy.linalg import sqrtm
def euclidean_distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2)**2))   
def sir_1(X, y, H, K):
    """Sliced Inverse Regression : Method 1
        Parameters
        ----------
        X : float, array of shape [n_samples, dimension]
            The generated samples.
        y : int, array of shape [n_samples, ]
            Labels of each generated sample.            
        y1 : repeating samples of y_{t+h}        
        H : number of slices
        K : return K estimated e.d.r directions
        Returns
        -------
        edr_est : estimated e.d.r directions
        """
    Z = X-np.mean(X, axis=0)
    width = (np.max(y) - np.min(y)) / H
    V_hat = np.zeros([X.shape[1], X.shape[1]])
    for h in range(H):
        h_index = np.logical_and(np.min(y)+h*width <= y, y < np.min(y)+(h+1)*width)
        ph_hat = np.mean(h_index)
        if ph_hat == 0:
            continue
        mh = np.mean(Z[h_index, :], axis=0)
        V_hat = np.add(V_hat,ph_hat * np.matmul(mh[:, np.newaxis], mh[np.newaxis, :]))
    eigenvalues, eigenvectors = np.linalg.eig(V_hat)
    K_index = np.argpartition(np.abs(eigenvalues), X.shape[1]-K) >= X.shape[1]-K
    K_largest_eigenvectors = eigenvectors[:, K_index]
    edr_est =  K_largest_eigenvectors       
    return  edr_est

def sir_1_time_series(X, y, H, K):
    
    Z = X-np.mean(X, axis=0)
    width = (np.max(y) - np.min(y)) / H
    V_hat = np.zeros([X.shape[1], X.shape[1]])
    for h in range(H):
        h_index = np.logical_and(np.min(y)+h*width <= y, y < np.min(y)+(h+1)*width)
        ph_hat = np.mean(h_index)
        if ph_hat == 0:
            continue
        mh = np.mean(Z[h_index, :], axis=0)
        V_hat = np.add(V_hat,ph_hat * np.matmul(mh[:, np.newaxis], mh[np.newaxis, :]))

    eigenvalues, eigenvectors = np.linalg.eig(V_hat)
    K_index = np.argpartition(np.abs(eigenvalues), X.shape[1]-K) >= X.shape[1]-K
    K_largest_eigenvectors = eigenvectors[:, K_index]
    edr_est =  K_largest_eigenvectors         
    return  (V_hat, edr_est, eigenvalues ** 2)

In [6]:
#from 0 to Q 
from tabulate import tabulate
def upper1(ar_coeff):
    num_N = 5
    n_obs = 10000
    noise = np.zeros((num_N, n_obs))
    n = 100
    H = 48
    P = 4
    K = 1
    S = 20
    hat = [np.zeros((P, 1)) for i in range(S)]
    # ar_coeff = [0.2, 0.2, 0.2, 0.2]
    
    for w in range(n):    
        for h in range(num_N):
            noise[h] = np.random.normal(0, 1, size=n_obs)  # Normally distributed noise
        ar_series = np.zeros((num_N, n_obs))
        for t in range(0, n_obs):
            ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1]  + noise[0][t]
            ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1]  + noise[1][t]
            ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1]  + noise[2][t]
            ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1]  + noise[3][t]
            ar_series[4][t] = 2*ar_series[0][t] + 3*ar_series[1][t] + noise[4][t]
        y = ar_series[4]
        X = np.concatenate([ar_series[i].reshape(-1,1) for i in range(4)], axis = 1)
        V = []
        for a in range(0,S+1):
            M, edr, lam1 = sir_1_time_series(X[:n_obs - a + 1], y[:n_obs - a + 1], H=H, K=K)
            V.append(M)
        for q in range(1, S+1):
            Q = np.zeros((P, P))
            phi = ar_coeff
            for j in range(P):
                for k in range(P):
                    Q[j,k] = sum(phi[j]**a * V[a][j,k] * phi[k]**a for a in range(0,q))  #phi[j] and phi[k]
            eigenvalues1, eigenvectors1 = np.linalg.eig(Q)
            K_index = np.argpartition(np.abs(eigenvalues1), P-K) >= P-K
            K_largest_eigenvectors = eigenvectors1[:, K_index]
            edr_est =  K_largest_eigenvectors
            if edr_est[0]<0:
                edr_est = -edr_est
            edr_est = edr_est/np.linalg.norm(edr_est)
            hat[q-1] += edr_est       
    for i in range(S):
        hat[i] = hat[i]/n

    array = np.array(hat)
    
    print(tabulate(array, tablefmt='latex'))
#yes

In [11]:
#from 0 to Q 
from tabulate import tabulate
ar_coeff = [0.2,0.2,0.2,0.2]
num_N = 5
n_obs = 10000
noise = np.zeros((num_N, n_obs))
n = 100
H = 48
P = 4
K = 1
S = 20
hat = [np.zeros((P, 1)) for i in range(S)]
# ar_coeff = [0.2, 0.2, 0.2, 0.2]

for w in range(n):    
    for h in range(num_N):
        noise[h] = np.random.normal(0, 1, size=n_obs)  # Normally distributed noise
    ar_series = np.zeros((num_N, n_obs))
    for t in range(0, n_obs):
        ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1]  + noise[0][t]
        ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1]  + noise[1][t]
        ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1]  + noise[2][t]
        ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1]  + noise[3][t]
        ar_series[4][t] = 2*ar_series[0][t] + 3*ar_series[1][t] + noise[4][t]
    y = ar_series[4]
    X = np.concatenate([ar_series[i].reshape(-1,1) for i in range(4)], axis = 1)
    V = []
    for a in range(0,S+1):
        M, edr, lam1 = sir_1_time_series(X[:n_obs - a + 1], y[:n_obs - a + 1], H=H, K=K)
        V.append(M)
    for q in range(1, S+1):
        Q = np.zeros((P, P))
        phi = ar_coeff
        for j in range(P):
            for k in range(P):
                Q[j,k] = sum(phi[j]**a * V[a][j,k] * phi[k]**a for a in range(0,q))  #phi[j] and phi[k]
        eigenvalues1, eigenvectors1 = np.linalg.eig(Q)
        K_index = np.argpartition(np.abs(eigenvalues1), P-K) >= P-K
        K_largest_eigenvectors = eigenvectors1[:, K_index]
        edr_est =  K_largest_eigenvectors
        if edr_est[0]<0:
            edr_est = -edr_est
        edr_est = edr_est/np.linalg.norm(edr_est)
        hat[q-1] += edr_est       
for i in range(S):
    hat[i] = hat[i]/n

array = np.array(hat)

print(tabulate(array, tablefmt='latex'))

\begin{tabular}{rrrr}
\hline
 0.556623 & 0.830532 & -0.00174234 & 0.000594235 \\
 0.556623 & 0.830532 & -0.00174234 & 0.000594235 \\
 0.556623 & 0.830532 & -0.00174234 & 0.00059423  \\
 0.556623 & 0.830532 & -0.00174234 & 0.00059423  \\
 0.556623 & 0.830532 & -0.00174234 & 0.00059423  \\
 0.556623 & 0.830532 & -0.00174234 & 0.00059423  \\
 0.556623 & 0.830532 & -0.00174234 & 0.00059423  \\
 0.556623 & 0.830532 & -0.00174234 & 0.00059423  \\
 0.556623 & 0.830532 & -0.00174234 & 0.00059423  \\
 0.556623 & 0.830532 & -0.00174234 & 0.00059423  \\
 0.556623 & 0.830532 & -0.00174234 & 0.00059423  \\
 0.556623 & 0.830532 & -0.00174234 & 0.00059423  \\
 0.556623 & 0.830532 & -0.00174234 & 0.00059423  \\
 0.556623 & 0.830532 & -0.00174234 & 0.00059423  \\
 0.556623 & 0.830532 & -0.00174234 & 0.00059423  \\
 0.556623 & 0.830532 & -0.00174234 & 0.00059423  \\
 0.556623 & 0.830532 & -0.00174234 & 0.00059423  \\
 0.556623 & 0.830532 & -0.00174234 & 0.00059423  \\
 0.556623 & 0.830532 & -0.00174234 

In [17]:
X.shape

(10000, 4)

In [7]:
ar_coeff = [0.2,0.2,0.2,0.2]
upper1(ar_coeff)

\begin{tabular}{rrrr}
\hline
 0.554829 & 0.831759 & -0.00155251 & 0.000934021 \\
 0.554829 & 0.831759 & -0.00155251 & 0.000934021 \\
 0.554829 & 0.831759 & -0.00155251 & 0.00093402  \\
 0.554829 & 0.831759 & -0.00155251 & 0.00093402  \\
 0.554829 & 0.831759 & -0.00155251 & 0.00093402  \\
 0.554829 & 0.831759 & -0.00155251 & 0.00093402  \\
 0.554829 & 0.831759 & -0.00155251 & 0.00093402  \\
 0.554829 & 0.831759 & -0.00155251 & 0.00093402  \\
 0.554829 & 0.831759 & -0.00155251 & 0.00093402  \\
 0.554829 & 0.831759 & -0.00155251 & 0.00093402  \\
 0.554829 & 0.831759 & -0.00155251 & 0.00093402  \\
 0.554829 & 0.831759 & -0.00155251 & 0.00093402  \\
 0.554829 & 0.831759 & -0.00155251 & 0.00093402  \\
 0.554829 & 0.831759 & -0.00155251 & 0.00093402  \\
 0.554829 & 0.831759 & -0.00155251 & 0.00093402  \\
 0.554829 & 0.831759 & -0.00155251 & 0.00093402  \\
 0.554829 & 0.831759 & -0.00155251 & 0.00093402  \\
 0.554829 & 0.831759 & -0.00155251 & 0.00093402  \\
 0.554829 & 0.831759 & -0.00155251 

In [8]:
ar_coeff = [0.2,0.2,0.8,0.8]
upper1(ar_coeff)

\begin{tabular}{rrrr}
\hline
 0.553566 & 0.8323   & 0.00323013 & 0.00013704  \\
 0.553514 & 0.832221 & 0.00362547 & 0.00015457  \\
 0.553501 & 0.832201 & 0.00371643 & 0.000158918 \\
 0.553498 & 0.832196 & 0.00374059 & 0.000160418 \\
 0.553496 & 0.832193 & 0.00375016 & 0.000161293 \\
 0.553495 & 0.832192 & 0.00375531 & 0.000161802 \\
 0.553495 & 0.832191 & 0.00375848 & 0.000162122 \\
 0.553495 & 0.832191 & 0.00376049 & 0.000162324 \\
 0.553494 & 0.83219  & 0.00376177 & 0.000162453 \\
 0.553494 & 0.83219  & 0.00376258 & 0.000162534 \\
 0.553494 & 0.83219  & 0.0037631  & 0.000162586 \\
 0.553494 & 0.83219  & 0.00376344 & 0.000162618 \\
 0.553494 & 0.83219  & 0.00376365 & 0.000162639 \\
 0.553494 & 0.83219  & 0.00376379 & 0.000162652 \\
 0.553494 & 0.83219  & 0.00376387 & 0.000162661 \\
 0.553494 & 0.83219  & 0.00376393 & 0.000162667 \\
 0.553494 & 0.83219  & 0.00376397 & 0.000162671 \\
 0.553494 & 0.83219  & 0.00376399 & 0.000162673 \\
 0.553494 & 0.83219  & 0.003764   & 0.000162675 \\
 0

In [9]:
ar_coeff = [0.2,0.5,0.8,0.8]
upper1(ar_coeff)

\begin{tabular}{rrrr}
\hline
 0.460802 & 0.886897 & -0.0012136  & -0.00184056 \\
 0.41971  & 0.906914 & -0.00138074 & -0.00207255 \\
 0.407655 & 0.912307 & -0.00147114 & -0.00219572 \\
 0.404405 & 0.913707 & -0.001516   & -0.00225602 \\
 0.40356  & 0.914059 & -0.00153742 & -0.00228431 \\
 0.403343 & 0.914145 & -0.00154733 & -0.00229734 \\
 0.403287 & 0.914165 & -0.00155208 & -0.00230335 \\
 0.403272 & 0.914169 & -0.00155441 & -0.00230619 \\
 0.403268 & 0.91417  & -0.0015556  & -0.00230759 \\
 0.403267 & 0.91417  & -0.00155625 & -0.00230832 \\
 0.403267 & 0.91417  & -0.00155662 & -0.00230872 \\
 0.403267 & 0.91417  & -0.00155683 & -0.00230894 \\
 0.403266 & 0.914169 & -0.00155696 & -0.00230908 \\
 0.403266 & 0.914169 & -0.00155704 & -0.00230916 \\
 0.403266 & 0.914169 & -0.00155709 & -0.00230921 \\
 0.403266 & 0.914169 & -0.00155712 & -0.00230924 \\
 0.403266 & 0.914169 & -0.00155714 & -0.00230926 \\
 0.403266 & 0.914169 & -0.00155715 & -0.00230928 \\
 0.403266 & 0.914169 & -0.00155716 